In [1]:
%matplotlib inline

In [2]:
from Bio import SeqIO, Phylo
import glob
import random

import pandas as pd
import numpy as np

# Just working with eukaryotes first where clusters are defined

In [3]:
monophyly_df = pd.read_csv('../Data/tree_monophyly.csv', index_col='OMA_group')
monophyly_df.head()

,monophyletic_clade,other_clade,root_bl,monophyletic_total_bl,other_total_bl,total_tree_bl,total_n,monophyletic_n,other_n
OMA_group,,,,,,,,,
OMAGroup_781946,IntNode_70,IntNode_97,0.49919,14.27866,13.28222,28.06007,218.0,98.0,120.0
OMAGroup_519799,IntNode_37,IntNode_56,0.95431,36.50749,42.45750,79.91930,182.0,57.0,125.0
OMAGroup_827022,IntNode_68,IntNode_69,0.74559,29.35498,28.03554,58.13611,194.0,70.0,124.0
OMAGroup_809653,IntNode_48,IntNode_68,0.27492,47.96207,48.17366,96.41065,208.0,89.0,119.0
OMAGroup_840979,IntNode_66,IntNode_93,0.79274,31.50017,44.09551,76.38842,218.0,94.0,124.0


In [5]:
fraction_to_prune = 0.9
for tree_loc in glob.glob('../Data/raw_OMA_trees/*.treefile.Rooted.MPAJH')[:]:
    print(tree_loc)
    file_name = tree_loc.split('/')[-1]
    oma_group = tree_loc.split('/')[-1].split('.')[0]
    if type(monophyly_df.loc[oma_group]['monophyletic_clade']) != str:
        continue
    starting_tree = Phylo.read(tree_loc, 'newick')
    starting_tree.root_with_outgroup(monophyly_df.loc[oma_group]['other_clade'], outgroup_branch_length=10e-10)
    starting_tree.root_with_outgroup(monophyly_df.loc[oma_group]['monophyletic_clade'], outgroup_branch_length=10e-10)
    assert starting_tree.root.clades[1].name == monophyly_df.loc[oma_group]['monophyletic_clade']
    assert starting_tree.root.clades[0].name == monophyly_df.loc[oma_group]['other_clade']
    initial_dists = [clade.branch_length for clade in starting_tree.root.clades]
    ########
    prune_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
    assert len(prune_ids) == len(starting_tree.root.clades[1].get_terminals())
#     prune_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
#     assert len(prune_ids) == len(starting_tree.root.clades[0].get_terminals())
    ########
    n_to_remove = int(fraction_to_prune*len(prune_ids))
    if n_to_remove == len(prune_ids):
        n_to_remove = len(prune_ids)-1
        print('almost removed them all!')
    ###I'm making this complicated here so that I don't ever accidentally remove the node entirely!
    for i in range(n_to_remove):
        #######
        l_terms = starting_tree.root.clades[1].clades[0].get_terminals()
        r_terms = starting_tree.root.clades[1].clades[1].get_terminals()
#         l_terms = starting_tree.root.clades[0].clades[0].get_terminals()
#         r_terms = starting_tree.root.clades[0].clades[1].get_terminals()
        #######
        if len(l_terms) > 1 and len(r_terms) > 1:
            combined_terms = l_terms + r_terms
        elif len(l_terms) > 1 and len(r_terms) == 1:
            combined_terms = l_terms
        elif len(l_terms) == 1 and len(r_terms) > 1:
            combined_terms = r_terms
        else:
            print('Something unexpected is happening')
            print(len(l_terms), len(r_terms))
        term_to_remove = random.choice(combined_terms)
        starting_tree.prune(term_to_remove)
    final_dists = [clade.branch_length for clade in starting_tree.root.clades]
    assert initial_dists == final_dists
    outfile_loc = tree_loc.replace('raw_OMA_trees/', 'pruned_OMA_trees/')
    outfile_loc = outfile_loc.replace(file_name, 
                                      '{}.pruned_{}_dikarya.treefile'.format(oma_group, fraction_to_prune))
    Phylo.write(starting_tree, outfile_loc, 'newick')

../Data/raw_OMA_trees/OMAGroup_781946.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_519799.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_827022.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_809653.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_840979.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_819074.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_733464.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_733878.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_830349.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_506216.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_652554.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_839159.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_555520.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_trees/OMAGroup_838979.mafft.afa.treefile.Rooted.MPAJH
../Data/raw_OMA_tree